# Aumento Limite Clientes Tauste Pay

## Definição de Parâmetros

In [1]:
data_bases = '2022-05-28' #yyyy-mm-dd
dias_cadastro = 180 #Quantidade de dias após cadasto para considerar cliente
dias_limite_fatura = 180 #Quantidade de dias para corte das faturas consideradas.
dias_alt_lim = 90 #Quantidade de dias desde a última alteração de limite
perc_gast_lim = 30 #Percentual de uso do limite para considerar cliente
tolerancia_dias_pgto = 10 #Pode pagar até 10 dias depois sem ser desconsiderado da base
tolerancia_perc_pgto = 2 #Pode pagar até 2% a menos da fatura sem ser desconsiderado da base
tolerancia_vlr_pgto = 10 #Pode pagar até R$10,00 a menos da fatura sem ser desconsiderado da base
qnt_max_fat = 6 #Quantidade máxima de faturas para serem consideradas
teto_lim = 4000 #Valor máximo de limite disponível
faixas_acrescimo = [ #   Mínimo -- Máximo -- Percentual de Acréscimo do Limite
    [0,1000,50],
    [1000,2000,40],
    [2000,3000,30],
    [3000,4000,20]
] 

pag_fat = 'base/01.05.01___Pagamentos_de_Faturas_por_Data.csv'
faturas = 'base/01.06.01___Faturas_por_Data_de_Corte_ou_Vencimento.csv'
clientes = 'base/01.02.01___Clientes_Cadastrados.csv'
alt_lim = 'base/01.01.07___Ocorrências_de_Alteração_de_Limite.csv'

filiais = {
    'LOJA SISTEMA':'000-MIAADM',
    'TAUSTE SUPERMERCADOS REPUBLICA':'001-MIAREP',
    'TAUSTE SUPERMERCADOS TIRADENTES':'002-MIATIR',
    'TAUSTE SUPERMERCADOS RIO BRANCO':'003-BRURBR',
    'TAUSTE SUPERMERCADOS CAMPOLIM':'004-SOCCAM',
    'TAUSTE SUPERMERCADOS ITAVUVU':'005-SOCITA',
    'TAUSTE SUPERMERCADOS OZANAN':'006-JUNOZA',
    'TAUSTE SUPERMERCADOS DUQUE':'007-BRUDUQ',
    'TAUSTE SUPERMERCADOS GENERAL':'008-SOCGEN'
}

## Preparação dos Dados

#### Funções Acessórias

In [2]:
import pandas as pd
import numpy as np

In [3]:
dt_corte_venc= pd.to_datetime(data_bases)-np.timedelta64(dias_limite_fatura,'D')

def format_header(df):
    df = df.rename(
            lambda x: (
                x
                .replace('Ó','O')
                .replace('Ã','A')
                .replace('Ç','C')
                .replace('Á','A')
                .replace('Ú','U')
                .replace('É','E')
                .replace('Ê','E')
                .replace('Í','I')
                .replace(' ','_')
            ), axis='columns'
    )
    return df
def convert_currency_value(df,cols):
    df[cols] = df[cols].apply(lambda x: (
            x
            .str.replace('R\$ ', '', regex=True)
            .str.replace('.', '', regex=False)  # Encoding: cp1252
            .str.replace(',', '.', regex=False) # Encoding: cp1252
            #.str.replace(',', '', regex=False) # Encoding: Padrão
            .astype(float)
        )
     )
    return df
def str_to_datetime(df,cols):
    df[cols] = df[cols].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
    return df
def nome_filial(df,cols):
    df['filial'] = df[cols].apply(lambda x: filiais[x])
    return df
def statusPgtoFat(df):
    def converter(vr_pago,vr_total):
        if pd.isna(vr_pago):
            return "NÃO PAGA"
        elif np.isclose(vr_pago,vr_total):
            return "PAGA"
        elif vr_pago > vr_total:
            return "PAGO A MAIS"
        elif (vr_pago >= (vr_total - tolerancia_vlr_pgto)) and (vr_pago >= (vr_total * (1 - (tolerancia_perc_pgto/100)))):
            return "TOLERANCIA"                                            
        else:
            return "PGTO PARCIAL"
    df['status_pgto'] = df.apply(lambda x: converter(x.valor_pago,x.valor_total),axis='columns')
    return df
def statusVencFat(df):
    def converter(dt_pgto,dt_venc):
        diffDays = (dt_pgto - dt_venc)/np.timedelta64(1,'D')
        if diffDays <= 0:
            return "NO PRAZO"
        elif diffDays > 0 and diffDays <= tolerancia_dias_pgto :
            return "ATRASADA"
        else:
            return "VENCIDA"
    df['status_venc'] = df.apply(lambda x:converter(x.data_pagamento,x.data_vencimento),axis='columns')
    return df
def calc_acrescimo(valor):
    for faixa in faixas_acrescimo:
        if valor >= faixa[0] and valor < faixa[1]:
            return faixa[2]
    return faixas_acrescimo[-1][2]
def sugerirNovoLim(df):
    def converter(vr_limite):
        sugestao = vr_limite * (1 + (calc_acrescimo(vr_limite)/100))
        if vr_limite >= teto_lim:
            return vr_limite
        if sugestao > teto_lim:
            return float(teto_lim)
        else:
            return round(sugestao,2)
    df['aumento_lim'] = df['vlr_limite'].apply(lambda x: converter(x))
    return df

#### Tratamento Base Pagamentos das faturas 

In [4]:
def tratamento_pag_fat(file_name):
    df = (
        pd.read_csv(file_name,sep=',',encoding='cp1252')
        .pipe(format_header)
        .rename(str.lower,axis='columns')
        .dropna(axis='rows',how='all',subset='cod_conta')
        .pipe(nome_filial, 'nome_fantasia')
        .pipe(str_to_datetime, ['data_pagamento'])
        .pipe(convert_currency_value, ['valor_pago'])
        .assign(
            cod_conta=lambda x: x.cod_conta.astype(int),
            cod_fatura=lambda x: x.cod_fatura.astype(int),
        )
        .loc[lambda x: x.status != 'CANCELADO']
        .groupby(by='cod_fatura',axis='rows').agg({'data_pagamento': 'max','valor_pago': 'sum'}).reset_index()
    )
    return df

#### Tratamento Base Faturas 

In [5]:
def tratamento_faturas(file_name):
    to_drop = [
        'produto',
        'nome',
        'cpf',  
        'telefone_celular',
        'data_corte'
    ]
    df = (
        pd.read_csv(file_name,sep=',',encoding='cp1252')
        .pipe(format_header)
        .rename(str.lower,axis='columns')
        .rename(columns={'codigo_conta':'cod_conta','codigo_fatura':'cod_fatura'})
        .dropna(axis='rows',how='all',subset='cod_conta')
        .pipe(convert_currency_value, ['principal','encargos','encargos_rolados','taxas','taxas_roladas','valor_total'])
        .pipe(str_to_datetime, ['data_vencimento'])
        .assign(
            cod_conta=lambda x: x.cod_conta.astype(int),
            cod_fatura=lambda x: x.cod_fatura.astype(int)
        )
        .drop(to_drop,axis='columns')
    )
    return df

#### Tratamento Base Clientes

In [6]:
def tratamento_clientes(file_name):
    to_drop = [
        'produto',
        'nome',
        'cpf',
        'cnpj',
        'rg',
        'e-mail',
        'telefone_resid.',
        'origem_comercial',
        'usuario_cadastro',
        'telefone_celular',
        'telefone_comercial',
        'logradouro',
        'numero',
        'complemento',
        'bairro',
        'localidade',
        'uf',
        'cep',
        'data_cadastro',
        'data_nascimento',
        'dia_corte',
        'dia_venc.'
    ]
    df = (
        pd.read_csv(file_name,sep=',',encoding='cp1252')
        .pipe(format_header)
        .rename(str.lower,axis='columns')
        .dropna(axis='rows',how='all',subset='cod_conta')
        .pipe(convert_currency_value, ['vlr_limite'])
        .pipe(str_to_datetime, ['data_cadastro','data_nascimento'])
        .assign(
            dias_cadastro=lambda x: ((pd.to_datetime(data_bases) - x.data_cadastro) / np.timedelta64(1,'D')).astype(int),
            idade=lambda x: ((pd.to_datetime(data_bases) - x.data_nascimento) / np.timedelta64(1,'Y')).astype(int)
        )
        .drop(to_drop,axis='columns')
    )
    return df

#### Tratamento Base Limite

In [7]:
def tratamento_alt_lim(file_name):
    to_drop = [
        'produto',
        'nome',
        'cpf',
        'login_usuario',
        'telefone_residencial',
        'origem_comercial',
        'telefone_celular',
        'logradouro',
        'numero',
        'complemento',
        'bairro',
        'cidade',
        'uf',
        'cep',
        'data_cadastro',
        'data_ocorrencia'
    ]
    df = (
        pd.read_csv(file_name,sep=',',encoding='cp1252')
        .pipe(format_header)
        .rename(str.lower,axis='columns')
        .rename(columns={'cod_cliente':'cod_conta'})
        .dropna(axis='rows',how='all',subset='cod_conta')
        .pipe(convert_currency_value, ['valor_renda','limite_atual','saldo_devedor','saldo_disponivel'])
        .pipe(str_to_datetime, ['data_ocorrencia'])
        .assign(
            dias_alt_lim=lambda x: ((pd.to_datetime(data_bases) - x.data_ocorrencia) / np.timedelta64(1,'D')).astype(int)
        )
        .drop(to_drop,axis='columns')
        .loc[:,['cod_conta','dias_alt_lim']]
    )
    return df

### Junção de todas as bases e filtros

In [8]:
def criarBase():
    df = (
        pd.merge(
            tratamento_faturas(faturas), 
            tratamento_pag_fat(pag_fat), 
            on='cod_fatura',
            how='left',
            validate='1:1'
        )
        .pipe(statusPgtoFat)
        .pipe(statusVencFat)
        .loc[
            lambda x: 
            ((pd.to_datetime(data_bases) - x['data_vencimento'])/np.timedelta64(1,'D') > tolerancia_dias_pgto) | (x['status_pgto'] == 'PAGA') | (x['status_pgto'] == "PAGO A MAIS") 
        ] #Desconsidera faturas que ainda podem ser pagas (estão dentro da tolerância). Também para não impactar no cáluclo de gasto médio.
        .assign(
            seq_fat= lambda x:
                x.sort_values(by=['data_vencimento'],ascending=False).groupby(['cod_conta'])['cod_fatura'].cumcount() + 1 #Adiciona contagem para as faturas de cada cliente 
        )
        .loc[
            lambda x:
            (x['seq_fat'] <= qnt_max_fat) & (x['data_vencimento'] >= dt_corte_venc) #Considera apenas as 6 últimas faturas ou faturas que venceram após a data de corte definida no início
        ]
        .loc[
            lambda x:
            ~x['cod_conta'].isin(x.loc[
                (x['status_venc'] == 'VENCIDA') | 
                (x['status_pgto'] == 'PGTO PARCIAL'),
                'cod_conta'
            ]) #Remove clientes que tiveram alguma fatura vencida ou paga parcialmente
        ]
        .groupby(by='cod_conta',axis='rows').agg({'valor_pago': 'sum','taxas':'sum','taxas_roladas':'sum','cod_fatura': 'count'}).reset_index()
        .rename(columns={'cod_fatura':'qt_fatura'})
        .assign(gasto_medio = lambda x: x.valor_pago/ x.qt_fatura) #Considera o valor pago pois clientes podem pagar faturas a mais com o objetivo de aumentar seu limite
        .merge(
            tratamento_clientes(clientes),
            on='cod_conta',
            how='left',
            validate='1:1'
        )
        .assign(perc_gast_lim= lambda x:round((x.gasto_medio / x.vlr_limite) * 100,2))
        .sort_values(by='perc_gast_lim',ascending=False)
        .loc[
            lambda x: 
            #(x.situacao ==  'ATIVO') &
                # Avaliar remoção do filtro de clientes ATIVOS, pois podem ocorrer casos de clients bloqueados (+3 dias úteis de atraso)
                # tais clientes ainda podem pagar a fatura mais recente dentro da tolerância
            (x.dias_cadastro >= dias_cadastro) &
            (x.perc_gast_lim >= perc_gast_lim)
        ]
        .sort_values('dias_cadastro',ascending=False)
        .merge(
            tratamento_alt_lim(alt_lim),
            on='cod_conta',
            how='left',
            validate='1:1'
        )
        .loc[
            lambda x:
            (x.dias_alt_lim >= dias_alt_lim) | (x.dias_alt_lim.isnull())
        ]
        .pipe(sugerirNovoLim)
        .sort_values('aumento_lim',ascending=False)
    )
    return df

## Exportação

In [9]:
df = criarBase()

In [10]:
def gerarExcel():
    with pd.ExcelWriter('aumento_limite_final.xlsx') as writer:  
        (
            df
            .sort_values('aumento_lim', ascending = False)
            .to_excel(writer, sheet_name='Alteração Limte',index=False)
        )
    return

def gerarCargaLimites():
    (
        df[['cod_conta','aumento_lim']]
        .rename(columns = {'cod_conta':'IdConta','aumento_lim':'NovoLimite'})
        .assign(NovoLimite = lambda x: (x.NovoLimite*100).astype(int).astype(str))
        .to_csv('carga_atualizacao_limite.txt', sep=';',index=False)
    )
    return

In [11]:
gerarCargaLimites()
gerarExcel()